In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from GPAT.gpat.detect import detect_and_track
from GPAT.gpat.pose_estimate_from_tracking_data import pose_estimate_from_tracking_data
from GPAT.gpat.utils.gpat2fpat import gpat2fpat

In [12]:
video_name = 'center_52'

In [13]:
video_path = f"/home/ohwada/sasaki_20240930/video/{video_name}.MP4"
det_model_path = "/home/ohwada/human_pose_estimation/models/rtmdet_l_8xb32-300e_coco.py"
det_config_path = "/home/ohwada/human_pose_estimation/models/rtmdet_l_8xb32-300e_coco_20220719_112030-5a0be7c4.pth"
pose_model_path = "/home/ohwada/human_pose_estimation/models/rtmpose-l_8xb32-270e_coco-wholebody-384x288.py"
pose_config_path = "/home/ohwada/human_pose_estimation/models/rtmpose-l_simcc-coco-wholebody_pt-aic-coco_270e-384x288-eaeb96c8_20230125.pth"
output_path = "/home/ohwada/sasaki_20240930/" 
tracking_data_path = f"{output_path}data/{video_name}/tracking_data.csv"

In [14]:
detect_and_track(
    video_path = video_path,
    model_path = det_model_path,
    config_path = det_config_path,
    output_path = output_path,
)

Using device: cuda
Loads checkpoint by local backend from path: /home/ohwada/human_pose_estimation/models/rtmdet_l_8xb32-300e_coco_20220719_112030-5a0be7c4.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

Processing frame: 421/420
Done


In [15]:
df = pd.read_csv(tracking_data_path)
df.head()

,frame,track_id,x1,y1,x2,y2
0,1,1,703,258,870,892
1,1,2,497,328,720,725
2,1,3,1255,436,1471,671
3,2,1,703,258,869,891
4,2,2,496,328,721,725


In [16]:
unique_ids = df['track_id'].unique()
max_distance = 0

for id in unique_ids:
    df_id = df[df['track_id'] == id]
    x1_min = df_id['x1'].min()
    y1_min = df_id['y1'].min()
    x1_max = df_id['x1'].max()
    y1_max = df_id['y1'].max()
    x2_min = df_id['x2'].min()
    y2_min = df_id['y2'].min()
    x2_max = df_id['x2'].max()
    y2_max = df_id['y2'].max()
    distance = np.sqrt((x1_max - x1_min)**2 + (y1_max - y1_min)**2 + (x2_max - x2_min)**2 + (y2_max - y2_min)**2)
    if distance > max_distance:
        max_distance = distance
        selected_id = id
        
track_id = selected_id
print(track_id)

1


In [17]:
pose_estimate_from_tracking_data(
    video_path = video_path,
    tracking_data_path = tracking_data_path,
    model_path = pose_model_path,
    config_path = pose_config_path,
    output_path = output_path,
    track_id = track_id,
)

Using device: cuda
Loads checkpoint by local backend from path: /home/ohwada/human_pose_estimation/models/rtmpose-l_simcc-coco-wholebody_pt-aic-coco_270e-384x288-eaeb96c8_20230125.pth
Processing frame: 421/420
Done


In [18]:
gpat2fpat(os.path.join(output_path, "data", video_name))